In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted = ld.getICMselected('7')
ICM_all = sps.hstack([ICM1, ICM3, ICML])
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=1234)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions


In [4]:
import os

ofp = "../models_temp/Similarity_Hybrid/"

models_to_combine_best = {
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' : {'num_factors': 34, 'epochs': 599, 'confidence_scaling': 'linear', 'alpha': 0.003519435539271083, 'epsilon': 0.09222402080721787, 'reg': 2.4127708108457617e-05},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True}
                         }


In [5]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [6]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid

In [7]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender

In [8]:
recommender_3 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
recommender_4 = SLIM_BPR_Cython(URM_train)
recommender_1 = RP3betaRecommender(URM_train)
recommender_2 = ItemKNNCBFRecommender(URM_train, ICM_weighted)
recommender3 = IALSRecommender(URM_train)

model_init(recommender_3, 'SLIM_Pure', models_to_combine_best['SLIMER'])
model_init(recommender_4, 'SLIM_BPR', models_to_combine_best['SLIMBPR'])
model_init(recommender_1, 'RP3beta', models_to_combine_best['RP3beta'])
model_init(recommender_2, 'KNNweigh', models_to_combine_best['icm_weighted'])
model_init(recommender3, 'IALS', models_to_combine_best['IALS'])

recommender1 = ItemKNNSimilarityHybridRecommenderNormal(URM_train, recommender_3.W_sparse, recommender_4.W_sparse)
recommender1.fit(0.9876793526315789)

recommender2 = ItemKNNSimilarityHybridRecommenderNormal(URM_train, recommender_1.W_sparse, recommender_2.W_sparse)
recommender2.fit(0.9546136842105264)

ItemKNNCBFRecommender: ICM Detected 22 ( 0.1%) items with no features.
../models_temp/Similarity_Hybrid/SLIM_Pure.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/SLIM_Pure'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/SLIM_BPR.zip
Model found!
SLIM_BPR_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/SLIM_BPR'
SLIM_BPR_Recommender: Loading complete
../models_temp/Similarity_Hybrid/RP3beta.zip
Model found!
RP3betaRecommender: Loading model from file '../models_temp/Similarity_Hybrid/RP3beta'
RP3betaRecommender: Loading complete
../models_temp/Similarity_Hybrid/KNNweigh.zip
Model found!
ItemKNNCBFRecommender: Loading model from file '../models_temp/Similarity_Hybrid/KNNweigh'
ItemKNNCBFRecommender: Loading complete
../models_temp/Similarity_Hybrid/IALS.zip
Model does not exists, creating...
IALSRecommender: Epoch 1 of 599. Elapsed time 32.20 sec
IALSRecommender: Epoch 2 of 599. Ela

IALSRecommender: Epoch 129 of 599. Elapsed time 2.17 hour
IALSRecommender: Epoch 130 of 599. Elapsed time 2.17 hour
IALSRecommender: Epoch 131 of 599. Elapsed time 2.17 hour
IALSRecommender: Epoch 132 of 599. Elapsed time 2.18 hour
IALSRecommender: Epoch 133 of 599. Elapsed time 2.18 hour
IALSRecommender: Epoch 134 of 599. Elapsed time 2.18 hour
IALSRecommender: Epoch 135 of 599. Elapsed time 2.18 hour
IALSRecommender: Epoch 136 of 599. Elapsed time 2.19 hour
IALSRecommender: Epoch 137 of 599. Elapsed time 2.19 hour
IALSRecommender: Epoch 138 of 599. Elapsed time 2.19 hour
IALSRecommender: Epoch 139 of 599. Elapsed time 2.19 hour
IALSRecommender: Epoch 140 of 599. Elapsed time 2.20 hour
IALSRecommender: Epoch 141 of 599. Elapsed time 2.20 hour
IALSRecommender: Epoch 142 of 599. Elapsed time 2.20 hour
IALSRecommender: Epoch 143 of 599. Elapsed time 2.20 hour
IALSRecommender: Epoch 144 of 599. Elapsed time 2.21 hour
IALSRecommender: Epoch 145 of 599. Elapsed time 2.21 hour
IALSRecommende

IALSRecommender: Epoch 271 of 599. Elapsed time 2.42 hour
IALSRecommender: Epoch 272 of 599. Elapsed time 2.42 hour
IALSRecommender: Epoch 273 of 599. Elapsed time 2.43 hour
IALSRecommender: Epoch 274 of 599. Elapsed time 2.43 hour
IALSRecommender: Epoch 275 of 599. Elapsed time 2.43 hour
IALSRecommender: Epoch 276 of 599. Elapsed time 2.43 hour
IALSRecommender: Epoch 277 of 599. Elapsed time 2.43 hour
IALSRecommender: Epoch 278 of 599. Elapsed time 2.43 hour
IALSRecommender: Epoch 279 of 599. Elapsed time 2.44 hour
IALSRecommender: Epoch 280 of 599. Elapsed time 2.44 hour
IALSRecommender: Epoch 281 of 599. Elapsed time 2.44 hour
IALSRecommender: Epoch 282 of 599. Elapsed time 2.44 hour
IALSRecommender: Epoch 283 of 599. Elapsed time 2.44 hour
IALSRecommender: Epoch 284 of 599. Elapsed time 2.45 hour
IALSRecommender: Epoch 285 of 599. Elapsed time 2.45 hour
IALSRecommender: Epoch 286 of 599. Elapsed time 2.45 hour
IALSRecommender: Epoch 287 of 599. Elapsed time 2.45 hour
IALSRecommende

IALSRecommender: Epoch 413 of 599. Elapsed time 2.68 hour
IALSRecommender: Epoch 414 of 599. Elapsed time 2.68 hour
IALSRecommender: Epoch 415 of 599. Elapsed time 2.68 hour
IALSRecommender: Epoch 416 of 599. Elapsed time 2.68 hour
IALSRecommender: Epoch 417 of 599. Elapsed time 2.68 hour
IALSRecommender: Epoch 418 of 599. Elapsed time 2.69 hour
IALSRecommender: Epoch 419 of 599. Elapsed time 2.69 hour
IALSRecommender: Epoch 420 of 599. Elapsed time 2.69 hour
IALSRecommender: Epoch 421 of 599. Elapsed time 2.69 hour
IALSRecommender: Epoch 422 of 599. Elapsed time 2.69 hour
IALSRecommender: Epoch 423 of 599. Elapsed time 2.69 hour
IALSRecommender: Epoch 424 of 599. Elapsed time 2.70 hour
IALSRecommender: Epoch 425 of 599. Elapsed time 2.70 hour
IALSRecommender: Epoch 426 of 599. Elapsed time 2.70 hour
IALSRecommender: Epoch 427 of 599. Elapsed time 2.70 hour
IALSRecommender: Epoch 428 of 599. Elapsed time 2.70 hour
IALSRecommender: Epoch 429 of 599. Elapsed time 2.71 hour
IALSRecommende

IALSRecommender: Epoch 555 of 599. Elapsed time 2.94 hour
IALSRecommender: Epoch 556 of 599. Elapsed time 2.94 hour
IALSRecommender: Epoch 557 of 599. Elapsed time 2.94 hour
IALSRecommender: Epoch 558 of 599. Elapsed time 2.94 hour
IALSRecommender: Epoch 559 of 599. Elapsed time 2.95 hour
IALSRecommender: Epoch 560 of 599. Elapsed time 2.95 hour
IALSRecommender: Epoch 561 of 599. Elapsed time 2.95 hour
IALSRecommender: Epoch 562 of 599. Elapsed time 2.95 hour
IALSRecommender: Epoch 563 of 599. Elapsed time 2.96 hour
IALSRecommender: Epoch 564 of 599. Elapsed time 2.96 hour
IALSRecommender: Epoch 565 of 599. Elapsed time 2.96 hour
IALSRecommender: Epoch 566 of 599. Elapsed time 2.96 hour
IALSRecommender: Epoch 567 of 599. Elapsed time 2.96 hour
IALSRecommender: Epoch 568 of 599. Elapsed time 2.97 hour
IALSRecommender: Epoch 569 of 599. Elapsed time 2.97 hour
IALSRecommender: Epoch 570 of 599. Elapsed time 2.97 hour
IALSRecommender: Epoch 571 of 599. Elapsed time 2.97 hour
IALSRecommende

In [9]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [10]:
rank_models(evaluator_validation, [recommender1, recommender2, recommender3])

EvaluatorHoldout: Processed 13643 (100.0%) in 48.63 sec. Users per second: 281
ItemKNNSimilarityHybridRecommender 0.25079121335553256
EvaluatorHoldout: Processed 13643 (100.0%) in 29.59 sec. Users per second: 461
ItemKNNSimilarityHybridRecommender 0.22369590661645253
EvaluatorHoldout: Processed 13643 (100.0%) in 6.03 sec. Users per second: 2262
IALSRecommender 0.2322584056711451


In [11]:
import matplotlib.pyplot as plt
import numpy as np

In [12]:
import numpy as np

def test_percentage(recommender_a, recommender_b, recommender_c, evaluator, high1, low1, high2, low2, high3, low3, step):
    recommender = ItemKNNScoresHybridMultipleRecommender(URM_train, recommender_a, recommender_b, recommender_c)
    results = []
    alp_space = np.linspace(high1, low1, step, True)
    beta_space = np.linspace(high2, low2, step, True)
    gamma_space = np.linspace(high3, low3, step, True)
    for alp in alp_space:
        for bet in beta_space:
            for gamma in gamma_space:
                recommender.fit(alp, bet, gamma)
                r_d, _ = evaluator.evaluateRecommender(recommender)
                print(alp, bet, gamma, ":", r_d.loc[10]['MAP'])
                results.append(r_d.loc[10]['MAP'])
    
    return alp_space, beta_space, gamma_space, results

In [13]:
alp_space, results = test_percentage(recommender1, recommender2, recommender3, evaluator_validation, 0.5, 1, 0.1, 0.3, 0.01, 0.2, 25)

_ = plt.figure(figsize=(16, 9))
plt.plot(results, alp_space, label='alpha')
plt.plot(results, beta_space, label='beta')
plt.plot(results, gamma_space, label='gamma')
plt.ylabel('MAP')
plt.xlabel('hyper')
plt.legend()
plt.show()

CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.819672131147541
ItemKNNSimilarityHybridRecommender with weight beta: 0.1639344262295082
IALSRecommender with weight gamma: 0.01639344262295082
EvaluatorHoldout: Processed 13643 (100.0%) in 1.30 min. Users per second: 174
0.5 0.1 0.01 : 0.25060531361510013
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.8091706001348618
ItemKNNSimilarityHybridRecommender with weight beta: 0.16183412002697237
IALSRecommender with weight gamma: 0.028995279838165883
EvaluatorHoldout: Processed 13643 (100.0%) in 1.31 min. Users per second: 174
0.5 0.1 0.017916666666666668 : 0.25056329253096854
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7989347536617842
ItemKNNSimilarityHybridRecommender with weight beta: 0.15978695073235685
IALSRecommender with weight gamma: 0.041278295605858856
EvaluatorHoldout: Processed 13643 (100.0%) in 1.27 min. Users per second: 179
0.5 0.1 

EvaluatorHoldout: Processed 13643 (100.0%) in 1.50 min. Users per second: 151
0.5 0.1 0.19208333333333336 : 0.249498559642774
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.625
ItemKNNSimilarityHybridRecommender with weight beta: 0.125
IALSRecommender with weight gamma: 0.25
EvaluatorHoldout: Processed 13643 (100.0%) in 1.59 min. Users per second: 143
0.5 0.1 0.2 : 0.2493575808979325
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.8086253369272236
ItemKNNSimilarityHybridRecommender with weight beta: 0.17520215633423178
IALSRecommender with weight gamma: 0.016172506738544472
EvaluatorHoldout: Processed 13643 (100.0%) in 1.51 min. Users per second: 150
0.5 0.10833333333333334 0.01 : 0.2505679172643928
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7984031936127743
ItemKNNSimilarityHybridRecommender with weight beta: 0.17298735861610112
IALSRecommender with weight gamma: 0.028609447771124414
Eva

EvaluatorHoldout: Processed 13643 (100.0%) in 1.14 min. Users per second: 199
0.5 0.10833333333333334 0.17625000000000002 : 0.24974354835144436
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6309148264984227
ItemKNNSimilarityHybridRecommender with weight beta: 0.13669821240799157
IALSRecommender with weight gamma: 0.23238696109358573
EvaluatorHoldout: Processed 13643 (100.0%) in 1.18 min. Users per second: 193
0.5 0.10833333333333334 0.1841666666666667 : 0.24959450977244116
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.62467464862051
ItemKNNSimilarityHybridRecommender with weight beta: 0.1353461738677772
IALSRecommender with weight gamma: 0.23997917751171263
EvaluatorHoldout: Processed 13643 (100.0%) in 1.12 min. Users per second: 204
0.5 0.10833333333333334 0.19208333333333336 : 0.2495079865365004
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6185567010309279
ItemKNNSimilarityHybridRecomm

EvaluatorHoldout: Processed 13643 (100.0%) in 1.05 min. Users per second: 217
0.5 0.11666666666666667 0.16041666666666668 : 0.2499531971160293
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6369426751592356
ItemKNNSimilarityHybridRecommender with weight beta: 0.14861995753715498
IALSRecommender with weight gamma: 0.21443736730360938
EvaluatorHoldout: Processed 13643 (100.0%) in 1.15 min. Users per second: 198
0.5 0.11666666666666667 0.16833333333333336 : 0.24985992584138308
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.630583289542827
ItemKNNSimilarityHybridRecommender with weight beta: 0.14713610089332632
IALSRecommender with weight gamma: 0.22228060956384657
EvaluatorHoldout: Processed 13643 (100.0%) in 1.14 min. Users per second: 199
0.5 0.11666666666666667 0.17625000000000002 : 0.24970591930974714
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6243496357960457
ItemKNNSimilarityHybridRec

EvaluatorHoldout: Processed 6000 (44.0%) in 5.20 min. Users per second: 19
EvaluatorHoldout: Processed 13643 (100.0%) in 6.90 min. Users per second: 33
0.5 0.125 0.15250000000000002 : 0.2501541897757929
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.636604774535809
ItemKNNSimilarityHybridRecommender with weight beta: 0.15915119363395225
IALSRecommender with weight gamma: 0.20424403183023876
EvaluatorHoldout: Processed 13643 (100.0%) in 1.86 min. Users per second: 123
0.5 0.125 0.16041666666666668 : 0.2500170504322836
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6302521008403361
ItemKNNSimilarityHybridRecommender with weight beta: 0.15756302521008403
IALSRecommender with weight gamma: 0.21218487394957986
EvaluatorHoldout: Processed 13643 (100.0%) in 1.58 min. Users per second: 144
0.5 0.125 0.16833333333333336 : 0.24985594391681887
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.624024960998

...

EvaluatorHoldout: Processed 13643 (100.0%) in 56.95 sec. Users per second: 240
0.5208333333333334 0.225 0.0575 : 0.2508251862400514
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6420133538777607
ItemKNNSimilarityHybridRecommender with weight beta: 0.2773497688751926
IALSRecommender with weight gamma: 0.08063687724704674
EvaluatorHoldout: Processed 13643 (100.0%) in 56.85 sec. Users per second: 240
0.5208333333333334 0.225 0.06541666666666666 : 0.25083421465045763
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6358087487283826
ItemKNNSimilarityHybridRecommender with weight beta: 0.2746693794506612
IALSRecommender with weight gamma: 0.08952187182095625
EvaluatorHoldout: Processed 13643 (100.0%) in 56.75 sec. Users per second: 240
0.5208333333333334 0.225 0.07333333333333333 : 0.25077623387306
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6297229219143576
ItemKNNSimilarityHybridRecommender wi

EvaluatorHoldout: Processed 13643 (100.0%) in 56.62 sec. Users per second: 241
0.5208333333333334 0.23333333333333334 0.04166666666666667 : 0.25086264367214384
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6480041472265423
ItemKNNSimilarityHybridRecommender with weight beta: 0.2903058579574909
IALSRecommender with weight gamma: 0.061689994815966834
EvaluatorHoldout: Processed 13643 (100.0%) in 57.05 sec. Users per second: 239
0.5208333333333334 0.23333333333333334 0.04958333333333334 : 0.25083186447146033
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6416837782340863
ItemKNNSimilarityHybridRecommender with weight beta: 0.2874743326488706
IALSRecommender with weight gamma: 0.07084188911704313
EvaluatorHoldout: Processed 13643 (100.0%) in 56.70 sec. Users per second: 241
0.5208333333333334 0.23333333333333334 0.0575 : 0.250802606488124
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.635485510

EvaluatorHoldout: Processed 13643 (100.0%) in 56.89 sec. Users per second: 240
0.5208333333333334 0.24166666666666667 0.017916666666666668 : 0.25048714440453856
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.660676532769556
ItemKNNSimilarityHybridRecommender with weight beta: 0.30655391120507397
IALSRecommender with weight gamma: 0.032769556025369975
EvaluatorHoldout: Processed 13643 (100.0%) in 57.01 sec. Users per second: 239
0.5208333333333334 0.24166666666666667 0.025833333333333333 : 0.25048516071384974
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6541077969649398
ItemKNNSimilarityHybridRecommender with weight beta: 0.30350601779173203
IALSRecommender with weight gamma: 0.042386185243328094
EvaluatorHoldout: Processed 13643 (100.0%) in 56.96 sec. Users per second: 240
0.5208333333333334 0.24166666666666667 0.03375 : 0.2507224240816562
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.647

EvaluatorHoldout: Processed 13643 (100.0%) in 57.04 sec. Users per second: 239
0.5208333333333334 0.24166666666666667 0.19208333333333336 : 0.2498153800600129
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.5411255411255411
ItemKNNSimilarityHybridRecommender with weight beta: 0.2510822510822511
IALSRecommender with weight gamma: 0.20779220779220778
EvaluatorHoldout: Processed 13643 (100.0%) in 56.92 sec. Users per second: 240
0.5208333333333334 0.24166666666666667 0.2 : 0.24960494887197393
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6670224119530417
ItemKNNSimilarityHybridRecommender with weight beta: 0.32017075773745995
IALSRecommender with weight gamma: 0.012806830309498399
EvaluatorHoldout: Processed 13643 (100.0%) in 57.09 sec. Users per second: 239
0.5208333333333334 0.25 0.01 : 0.2503673783520596
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6603275224511358
ItemKNNSimilarityHybridR

KeyboardInterrupt: 